# NYC Energy Load and Weather Analysis

This notebook analyzes the relationship between weather conditions and energy load in New York City.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from pathlib import Path

# Set up plotting style
#plt.style.use('seaborn')
#%matplotlib inline

## 1. Data Loading and Preprocessing

First, we'll load our weather and energy load data from CSV files.

In [ ]:
# Filepaths
base_dir = Path.cwd()
weather_fp = str(base_dir / "NYCWeatherData/New York Weather Data.csv")
load_fp = str(base_dir / "NYCEnergyLoad/OASIS_Real_Time_Dispatch_Actual_Load.csv")

# Load weather data
weather_raw = pd.read_csv(
    weather_fp,
    parse_dates=["DATE"],
    low_memory=False
)

### Process Weather Data

In [ ]:
# Extract daily weather summaries
mask = weather_raw["DailyAverageDryBulbTemperature"].notna()
weather_daily = weather_raw.loc[mask, [
    "DATE",
    "DailyAverageDryBulbTemperature",
    "DailyAverageRelativeHumidity"
]].rename(columns={
    "DailyAverageDryBulbTemperature": "avg_temp_f",
    "DailyAverageRelativeHumidity":  "avg_humidity_pct"
})

weather_daily["date"] = weather_daily["DATE"].dt.date
weather_daily = weather_daily.groupby("date", as_index=True).first()
weather_daily.index = pd.to_datetime(weather_daily.index)

### Process Load Data

In [ ]:
# Load and process NYISO load data
load = pd.read_csv(
    load_fp,
    parse_dates=["RTD End Time Stamp"],
    usecols=["RTD End Time Stamp","RTD Actual Load"]
).rename(columns={"RTD End Time Stamp":"datetime","RTD Actual Load":"load_mw"})

load["date"] = load["datetime"].dt.date
load_daily = load.groupby("date", as_index=True)["load_mw"].max()
load_daily.index = pd.to_datetime(load_daily.index)

## 2. Data Integration and Feature Engineering

In [ ]:
# Merge weather and load data
df_daily = pd.concat([
    weather_daily[["avg_temp_f","avg_humidity_pct"]],
    load_daily.rename("daily_max_load_mw")
], axis=1, join="inner")

# Reset index and add weekend/holiday features
df_daily = df_daily.reset_index().rename(columns={"index":"date"})
df_daily["is_weekend"] = df_daily["date"].dt.weekday >= 5
us_holidays = holidays.US(years=df_daily["date"].dt.year.unique().tolist())
df_daily["is_holiday"] = df_daily["date"].isin(us_holidays)
df_daily = pd.get_dummies(df_daily, columns=["is_weekend", "is_holiday"], drop_first=True)

## 3. Basic Statistics

In [ ]:
# Calculate key statistics
stats = {
    "Max daily load": df_daily["daily_max_load_mw"].max(),
    "Max daily temperature": df_daily["avg_temp_f"].max(),
    "Max daily humidity": df_daily["avg_humidity_pct"].max(),
    "Average daily temperature": df_daily["avg_temp_f"].mean(),
    "Average daily humidity": df_daily["avg_humidity_pct"].mean(),
    "Average daily load": df_daily["daily_max_load_mw"].mean(),
    "Weekend average load": df_daily[df_daily["is_weekend_True"]]["daily_max_load_mw"].mean(),
    "Holiday average load": df_daily[df_daily["is_holiday_True"]]["daily_max_load_mw"].mean()
}

for key, value in stats.items():
    print(f"{key}: {value:.2f}")

## 4. Data Quality Check and Cleaning

In [ ]:
# Check for missing values
print("Missing values per column:\n", df_daily.isna().sum())

# Drop any days with missing data
df_daily = df_daily.dropna()

# Handle outliers using IQR method
for col in ["avg_temp_f", "avg_humidity_pct", "daily_max_load_mw"]:
    Q1 = df_daily[col].quantile(0.25)
    Q3 = df_daily[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    print(f"{col}: clipping to [{lower:.1f}, {upper:.1f}]")
    df_daily[col] = df_daily[col].clip(lower, upper)

## 5. Exploratory Data Analysis

In [ ]:
# Time series plot
fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(df_daily["date"], df_daily["daily_max_load_mw"], label="Daily Max Load (MW)")
ax2 = ax1.twinx()
ax2.plot(df_daily["date"], df_daily["avg_temp_f"], color="orange", label="Avg Temp (°F)")
ax1.set_xlabel("Date")
ax1.set_ylabel("Load (MW)")
ax2.set_ylabel("Temp (°F)")
fig.suptitle("Daily Peak Load vs. Average Temperature")
fig.legend(loc="upper left", bbox_to_anchor=(0.1,0.9))
plt.show()

In [ ]:
# Correlation analysis
sns.pairplot(df_daily, vars=["avg_temp_f","avg_humidity_pct","daily_max_load_mw"])
plt.suptitle("Pairplot of Weather vs. Load", y=1.02)
plt.show()

corr = df_daily[["avg_temp_f","avg_humidity_pct","daily_max_load_mw"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

## 6. Time Series Decomposition

In [ ]:
# Decompose the time series
ts = df_daily.set_index("date")["daily_max_load_mw"]
decomp = seasonal_decompose(ts, model="additive", period=7)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
decomp.trend.plot(ax=ax1, title="Trend Component")
decomp.seasonal.plot(ax=ax2, title="Seasonal Component (Weekly)")
decomp.resid.plot(ax=ax3, title="Residual Component")
plt.tight_layout()
plt.show()

## 7. Linear Regression Analysis

In [ ]:
# Fit linear regression model
X = df_daily[["avg_temp_f","avg_humidity_pct"]]
y = df_daily["daily_max_load_mw"]

lr = LinearRegression().fit(X, y)
print("Regression coefficients:", dict(zip(X.columns, lr.coef_)))
print("Intercept:", lr.intercept_)
print("R² score:", lr.score(X,y))

# Plot regression line
temp_range = np.linspace(X["avg_temp_f"].min(), X["avg_temp_f"].max(), 100)
humid_mean = X["avg_humidity_pct"].mean()
y_pred = lr.intercept_ + lr.coef_[0]*temp_range + lr.coef_[1]*humid_mean

plt.figure(figsize=(8,4))
plt.scatter(df_daily["avg_temp_f"], y, alpha=0.6, label="Actual")
plt.plot(temp_range, y_pred, color="red", label="Fit @ avg humidity")
plt.xlabel("Avg Temp (°F)")
plt.ylabel("Daily Max Load (MW)")
plt.title("Linear Fit: Load vs. Temp")
plt.legend()
plt.show()